# pip installs

In [ ]:
!mamba install pytorch=1.12 torchvision torchaudio cudatoolkit=11.3 -c pytorch -y -q

In [2]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.12.0+cu113.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.12.0+cu113.html
!pip install torch-geometric

Looking in links: https://data.pyg.org/whl/torch-1.12.0+cu113.html
  Using cached https://data.pyg.org/whl/torch-1.12.0%2Bcu113/torch_scatter-2.1.0%2Bpt112cu113-cp39-cp39-linux_x86_64.whl (8.9 MB)
Looking in links: https://data.pyg.org/whl/torch-1.12.0+cu113.html
  Using cached https://data.pyg.org/whl/torch-1.12.0%2Bcu113/torch_sparse-0.6.15%2Bpt112cu113-cp39-cp39-linux_x86_64.whl (3.5 MB)
  Using cached torch_geometric-2.1.0.post1-py3-none-any.whl


In [3]:
!mamba install -c conda-forge pyts -q -y

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


In [4]:
!pip install llvmpy
!pip install cython
!pip install numba
!pip install pandas
!pip install networkx
!pip install matplotlib
!pip install ts2vg
!pip install pytorch_lightning
!pip install tensorflow

  Using cached llvmpy-0.12.7.tar.gz (657 kB)
  Preparing metadata (setup.py) ... error
  ERROR: Command errored out with exit status 1:
   command: /opt/conda/bin/python3.9 -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-njbhsx6n/llvmpy_1cc141d8a23f4810ab097dd42e590db9/setup.py'"'"'; __file__='"'"'/tmp/pip-install-njbhsx6n/llvmpy_1cc141d8a23f4810ab097dd42e590db9/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /tmp/pip-pip-egg-info-rcld9iig
       cwd: /tmp/pip-install-njbhsx6n/llvmpy_1cc141d8a23f4810ab097dd42e590db9/
  Complete output (2 lines):
  Error: could not invoke ['llvm-config', '--version']
  Try setting LLVM_CONFIG_PATH=/path/to/llvm-config
  ----------------------------------------
  Using cached llvm

# Main code

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
import warnings
import sklearn
import os.path as osp
import matplotlib.pyplot as plt
import ts2vg
import pytorch_lightning as pl
import tensorflow as tf
import datetime

from pyts.image import MarkovTransitionField

from torch.nn import Linear, CrossEntropyLoss, BCEWithLogitsLoss
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import global_mean_pool, global_add_pool, global_max_pool, ChebConv, global_sort_pool
from torch_geometric.loader import DataLoader
from torch.nn import Sequential, BatchNorm1d, ReLU, Dropout
from torch_geometric.nn import GCNConv, GINConv, GINEConv, GATv2Conv, GATConv

from tqdm import tqdm

from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import f1_score
from sklearn.manifold import TSNE

from torchvision import transforms

from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import Callback
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks import LearningRateFinder
from pytorch_lightning.callbacks import BatchSizeFinder
from pytorch_lightning.callbacks import ProgressBarBase
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.loggers import CSVLogger

from ts2vg import NaturalVG
from ts2vg import HorizontalVG

2022-11-29 13:20:07.919384: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-29 13:20:08.539025: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-29 13:20:08.539126: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-11-29 13:20:08.539133: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
#all in one
def create_graph():
    warnings.filterwarnings("ignore")
    # preparation for un/cut graphs
    if len_type == "un/cut":
    
        df = pd.read_csv(path_main)  
        del df['Unnamed: 0']
        df.index, df.columns = [range(df.index.size), range(df.columns.size)]
        length_rss = int((df.columns.stop-2)/2)
        
        X = df.loc[:,df.columns[:length_rss]].to_numpy()
        Y = df[length_rss+1].to_numpy(dtype=np.uint8)
        X_mask = df.loc[:,df.columns[length_rss+2:]].to_numpy()
        
        if graph_type in ("MTF", "visual_on_MTF", "MTF_on_visual"):
            MTF = MarkovTransitionField(n_bins=length_rss)
            X_gaf = MTF.fit_transform(X)
        
        
    # preparation for random graphs
    elif len_type == "random":
        dataset_rss = np.load(path_main, allow_pickle=True)['arr_0']
        dataset_properties = np.load(path_properties, allow_pickle=True)['arr_0']
        dataset_mask = np.load(path_mask, allow_pickle=True)['arr_0']

        for i in range(len(dataset_properties)):
            dataset_properties[i,1] = int(dataset_properties[i,1])
        
        X = dataset_rss
        X_mask = dataset_mask
        Y = dataset_properties[:,2]
        Y_len = dataset_properties[:,0]
        if graph_type in ("MTF", "visual_on_MTF", "MTF_on_visual"):
            X_gaf = []
            for i in range(len(Y_len)):
                MTF = MarkovTransitionField(n_bins=Y_len[i])
                X_gaf_temp = MTF.fit_transform(X[i].reshape(1, -1))
                X_gaf.append(X_gaf_temp[0])
    
    # output will have all graphs 
    output = []
    
    # setting class_weights for graph
    global class_weights
    weight_param = Y
    if classif == "graph" and len_type == "un/cut":
        weight_param = Y
    class_weights = torch.tensor(class_weight.compute_class_weight(class_weight='balanced',
                                                                   classes=np.unique(weight_param),
                                                                   y=weight_param))
    
    # function for creating edge index and edge weight for a given MTF matrix
    def adjToEdgidx_MTF(adj_mat):
        edge_index = torch.from_numpy(adj_mat).nonzero().t().contiguous()
        row, col = edge_index
        edge_weight = adj_mat[row, col]#adj_mat[row, col]
        return edge_index, edge_weight
    
    def adjToEdgidx_visual(X_current):
        g = NaturalVG(weighted='distance')
        # g = HorizontalVG(weighted='distance')
        g.build(X_current)

        adj_mat_visual = np.zeros([len(X_current),len(X_current)], dtype='float')
        for i in range(len(g.edges)):
            x, y, q =g.edges[i]
            adj_mat_visual[x,y] = q
            adj_mat_visual[y,x] = q
        
        edge_index = torch.from_numpy(adj_mat_visual).nonzero().t().contiguous()
        row, col = edge_index
        edge_weight = adj_mat_visual[row, col]
        
        return edge_index, edge_weight
    
    def adjToEdgidx_join(adj_mat_MTF,X_current):
        g = NaturalVG(weighted='distance')
        #g = HorizontalVG(weighted='distance')
        g.build(X_current)
        
        #create matrix for visual
        adj_mat_visual = np.zeros([len(adj_mat_MTF),len(adj_mat_MTF)], dtype='float')
        for i in range(len(g.edges)):
            x, y, q =g.edges[i]
            adj_mat_visual[x,y] = q
            adj_mat_visual[y,x] = q
            
        if graph_type == "visual_on_MTF":
            temp_main_adj_mat = adj_mat_MTF
        elif graph_type == "MTF_on_visual":
            temp_main_adj_mat = adj_mat_visual
            
        edge_index = torch.from_numpy(temp_main_adj_mat).nonzero().t().contiguous()
        
        #join two edge_weight arrays (visual is converted to fit MTF) 
        row, col = edge_index
        edge_weight = np.zeros([len(row),2], dtype='float')
        edge_weight[:,0] = adj_mat_MTF[row, col]
        edge_weight[:,1] = adj_mat_visual[row, col]
        
        return edge_index, edge_weight
    
    def define_mask(i):
        if classif == "graph": # for graph classification
            return torch.tensor(Y[i], dtype=torch.long)      
        elif classif == "node":# for node classification 
            return torch.unsqueeze(torch.tensor(X_mask[i], dtype=torch.double),1)
        
    if graph_type == "MTF":  
        for i, j in enumerate(X_gaf):
            edge_index, edge_weight = adjToEdgidx_MTF(j)
            y_mask = define_mask(i)
            #Into Data save node values "x", edge index from adjacency matrix and edge features/attributes, finally label
            output.append(Data(x=torch.unsqueeze(torch.tensor(X[i], dtype=torch.double),1), edge_index=edge_index, edge_attr=torch.unsqueeze(torch.tensor(edge_weight, dtype=torch.double),1), y=y_mask))
    elif graph_type == "visual":
        for i in range(len(X)):
            edge_index, edge_weight = adjToEdgidx_visual(X[i])
            y_mask = define_mask(i)
            output.append(Data(x=torch.unsqueeze(torch.tensor(X[i], dtype=torch.double),1), edge_index=torch.tensor(edge_index, dtype=torch.int64), edge_attr=torch.unsqueeze(torch.tensor(edge_weight, dtype=torch.double),1),y=y_mask))    
    elif graph_type in ("visual_on_MTF", "MTF_on_visual"):
        for i, j in enumerate(X_gaf):
            edge_index, edge_weight = adjToEdgidx_join(j, X[i])
            y_mask = define_mask(i) 
            output.append(Data(x=torch.unsqueeze(torch.tensor(X[i], dtype=torch.double),1), edge_index=edge_index, edge_attr=torch.tensor(edge_weight, dtype=torch.double), y=y_mask))    
    return output

In [3]:
#keeps the last output so we dont have to generate another if all parameters are the same as before
global temp_repeat
temp_repeat=['']*6
def generate_output():
    if temp_repeat[0] != graph_type or temp_repeat[1] != classif or temp_repeat[2] != len_type or temp_repeat[3] != path_main or temp_repeat[4] != path_properties or temp_repeat[5] != path_mask:
    
        global output
        output = create_graph()
            
    temp_repeat[0] = graph_type 
    temp_repeat[1] = classif
    temp_repeat[2] = len_type
    temp_repeat[3] = path_main
    temp_repeat[4] = path_properties
    temp_repeat[5] = path_mask
    
    print(temp_repeat[1] + " classification on " + temp_repeat[2] + " time series using " + temp_repeat[0] + " graphs" )
    
    return output

def check_for_missclick():
    return graph_type in ("visual", "MTF", "MTF_on_visual", "visual_on_MTF") and classif in ("graph", "node") and len_type in ("un/cut", "random")
    

In [4]:
def my_test_loop():
    
    import os, sys
    versions = os.listdir("Models/"+version)
    len_ver = len(versions)
    SEED_temp=SEED
    for i in range(len_ver):
        
        global device, model
            
        torch.manual_seed(SEED_temp)
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        output = generate_output()

        train_size = int(0.8 * len(output))
        Temp_size = len(output) - train_size
        val_size = int(0.2*Temp_size)
        test_size = Temp_size - val_size
        train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(output, [train_size, val_size, test_size])

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

        model = GINE.load_from_checkpoint("Models/"+version+"/version_" + str(i) + "/checkpoints/" + monitor_test).double()
        #model = Net.load_from_checkpoint("tb_logs/impact_of_TS_length_on_GNN_classification_performance/version_41/checkpoints/best.ckpt").double()
        trainer = pl.Trainer(accelerator='gpu',devices=1)
        trainer.test(model, test_loader)
        print("version_"+str(i))
        print(SEED_temp)
        SEED_temp += 1

class FineTuneLearningRateFinder(LearningRateFinder):
    def __init__(self, milestones, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.milestones = milestones

    def on_fit_start(self, *args, **kwargs):
        return

    def on_train_epoch_start(self, trainer, pl_module):
        if trainer.current_epoch in self.milestones or trainer.current_epoch == 0:
            self.lr_find(trainer, pl_module)

# class for node classification            
class GINE(pl.LightningModule):
    def __init__(self):
        super(GINE, self).__init__()
        
        if graph_type in ("MTF", "visual"):
            edge_dim = 1
        elif graph_type in ("MTF_on_visual", "visual_on_MTF"):
            edge_dim = 2
            
        dim_h = 32
    
        self.conv1 = GINEConv(
            Sequential(Linear(dim_h, dim_h),
                       BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()), edge_dim=edge_dim)
        
        self.conv2 = GINEConv(
            Sequential(Linear(dim_h, dim_h), BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()), edge_dim=edge_dim)
        
        self.conv3 = GINEConv(
            Sequential(Linear(dim_h, dim_h), BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()), edge_dim=edge_dim)
        
        self.conv4 = GINEConv(
            Sequential(Linear(dim_h, dim_h), BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()), edge_dim=edge_dim)
        
        self.conv5 = GINEConv(
            Sequential(Linear(dim_h, dim_h), BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()), edge_dim=edge_dim)
        
        
        self.lin1 = Linear(dim_h*5, dim_h*5)
        self.lin2 = Linear(dim_h*5, 5)

    def forward(self, data):
        x, edge_index, edge_weight, batch = data.x, data.edge_index, data.edge_attr, data.batch
        
        # Node embeddings 
        h1 = self.conv1(x, edge_index, edge_attr=edge_weight)
        h2 = self.conv2(h1, edge_index, edge_attr=edge_weight)
        h3 = self.conv3(h2, edge_index, edge_attr=edge_weight)
        h4 = self.conv4(h3, edge_index, edge_attr=edge_weight)
        h5 = self.conv5(h4, edge_index, edge_attr=edge_weight)
        
        # Graph-level readout
        
        h1 = global_max_pool(h1, batch)
        h2 = global_max_pool(h2, batch)
        h3 = global_max_pool(h3, batch)
        h4 = global_max_pool(h4, batch)
        h5 = global_max_pool(h5, batch)
        
        # Concatenate graph embeddings
        h = torch.cat((h1, h2, h3, h4, h5), dim=1)

        # Classifier
        h = self.lin1(h)
        h = h.relu()
        h = F.dropout(h, p=0.5, training=self.training)
        h = self.lin2(h)
        
        return h
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=5e-4)
        return optimizer
    
    def training_step(self, train_batch, batch_idx):
                     
        out = model(train_batch)
        loss_function = CrossEntropyLoss(weight=class_weights).to(device) #weight=class_weights
        train_loss = loss_function(out, train_batch.y)
        
        correct=out.argmax(dim=1).eq(train_batch.y).sum().item()
        logs={"train_loss": train_loss}
        total=len(train_batch.y)
        
        batch_dictionary={"loss": train_loss, "log": logs, "correct": correct, "total": total}
        
        return train_loss
    
    
    def validation_step(self, val_batch, batch_idx):
      
        out = model(val_batch)
        loss_function = CrossEntropyLoss(weight=class_weights).to(device)
        val_loss = loss_function(out, val_batch.y)
        
        pred = out.argmax(-1)
        correct=out.argmax(dim=1).eq(val_batch.y).sum().item()
        total=len(val_batch.y)
        val_label = val_batch.y
        accuracy = (pred == val_label).sum() / pred.shape[0]
        
        logs={"train_loss": val_loss}
        batch_dictionary={"loss": val_loss, "log": logs, "correct": correct, "total": total}
        self.log("val_loss", val_loss)
        self.log("val_acc", accuracy)
        
    
    def test_step(self, test_batch, batch_idx):
        out = model(test_batch)
        loss_function = CrossEntropyLoss(weight=class_weights).to(device)
        test_loss = loss_function(out, test_batch.y)
        
        pred = out.argmax(-1)
        test_label = test_batch.y
        accuracy = (pred == test_label).sum() / pred.shape[0]
        self.log("test_true", test_label)
        self.log("test_pred", pred)
        self.log("test_acc", accuracy)
        return pred, test_label
        
    def test_epoch_end(self, outputs):
        #this function gives us in the outputs all acumulated pred and test_labels we returned in test_step
        #we transform the pred and test_label into a shape that the classification report can read
        true_array=[]
        pred_array = []
        for i in range(len(outputs)):
            true_array = np.append(true_array,outputs[i][1].cpu().numpy())
            pred_array = np.append(pred_array,outputs[i][0].cpu().numpy())            
        print(confusion_matrix(true_array, pred_array))
        print(classification_report(true_array, pred_array))
        return pred_array, true_array
    
#---------------------------------------------------------------------------------
# class for node classification
class Net(pl.LightningModule):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = GATConv(1, 32, heads=4)
        self.lin1 = torch.nn.Linear(1, 4 * 32)
        self.conv2 = GATConv(4 * 32, 32, heads=4)
        self.lin2 = torch.nn.Linear(4 * 32, 4 * 32)
        self.conv3 = GATConv(4 * 32, 1, heads=6,concat=False)
        self.lin3 = torch.nn.Linear(4 * 32, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        x = F.elu(self.conv1(x, edge_index) + self.lin1(x))
        x = F.elu(self.conv2(x, edge_index) + self.lin2(x))
        x = self.conv3(x, edge_index) + self.lin3(x)
        return x
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=5e-4)
        return optimizer
    


    def training_step(self, train_batch, batch_idx):        
        out = model(train_batch)
        loss_function = BCEWithLogitsLoss().to(device) #weight=class_weights[1]
        
        train_loss = loss_function(out, train_batch.y)
        correct=out.argmax(dim=1).eq(train_batch.y).sum().item()
        logs={"train_loss": train_loss}
        total=len(train_batch.y)
        
        batch_dictionary={"loss": train_loss, "log": logs, "correct": correct, "total": total}
        
        return train_loss
    
    
    def validation_step(self, val_batch, batch_idx):
      
        out = model(val_batch)
        loss_function = BCEWithLogitsLoss().to(device)
        val_loss = loss_function(out, val_batch.y)
        
        ys, preds = [], []
        val_label = val_batch.y.cpu()
        ys.append(val_batch.y)
        preds.append((out > 0).float().cpu())     
        y, pred = torch.cat(ys, dim=0), torch.cat(preds, dim=0)
        accuracy = (pred == val_label).sum() / pred.shape[0]
    
        self.log("val_loss", val_loss)
        self.log("val_acc", accuracy)
    
    def test_step(self, test_batch, batch_idx):
        # this is the test loop
        out = model(test_batch)
        loss_function = BCEWithLogitsLoss().to(device)
        test_loss = loss_function(out, test_batch.y)
        
        ys, preds = [], []
        test_label = test_batch.y.cpu()
        ys.append(test_batch.y)
        preds.append((out > 0).float().cpu())
        
        y, pred = torch.cat(ys, dim=0), torch.cat(preds, dim=0)
        accuracy = (pred == test_label).sum() / pred.shape[0]
        
        self.log("test_acc", accuracy)
        return pred, y
        
    def test_epoch_end(self, outputs):
        #this function gives us in the outputs all acumulated pred and test_labels we returned in test_step
        #we transform the pred and test_label into a shape that the classification report can read
        true_array=[]
        pred_array = []
        for i in range(len(outputs)):
            true_array = np.append(true_array,outputs[i][1].cpu().numpy())
            pred_array = np.append(pred_array,outputs[i][0].cpu().numpy())   
        print(confusion_matrix(true_array, pred_array))
        print(classification_report(true_array, pred_array))
        print("pred_array ",pred_array)

        
def main():
    if check_for_missclick() != True:
        print("one of the main parameters does not match")
    else:
        global model, test_loader, device
        
        early_stop = EarlyStopping(monitor='val_acc',patience=patience, strict=False,verbose=False, mode='max')
        val_checkpoint_acc = ModelCheckpoint(filename="max_acc-{epoch}-{step}-{val_acc:.3f}", monitor = "val_acc", mode="max")
        val_checkpoint_best_loss = ModelCheckpoint(filename="best_loss", monitor = "val_loss", mode="max")
        val_checkpoint_best_acc = ModelCheckpoint(filename="best_acc", monitor = "val_acc", mode="max")
        val_checkpoint_loss = ModelCheckpoint(filename="min_loss-{epoch}-{step}-{val_loss:.3f}", monitor = "val_loss", mode="min")
        latest_checkpoint = ModelCheckpoint(filename="latest-{epoch}-{step}", monitor = "step", mode="max",every_n_train_steps = 500,save_top_k = 1)
        batchsizefinder = BatchSizeFinder(mode='power', steps_per_trial=3, init_val=2, max_trials=25, batch_arg_name='batch_size')
        lr_finder = FineTuneLearningRateFinder(milestones=(5,10))
        
        torch.manual_seed(SEED)
        print(SEED)
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        output = generate_output()
        # my data        
        
        train_size = int(0.8 * len(output))
        Temp_size = len(output) - train_size
        val_size = int(0.2*Temp_size)
        test_size = Temp_size - val_size
        train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(output, [train_size, val_size, test_size])
                
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

        # mode
        if classif == "graph":
            model = GINE().double()#.to(device) #GINE is for graph classification
        elif classif == "node":
            model = Net().double()#.to(device) #net is for node classification
            
        #training
        logger = TensorBoardLogger(save_file, name=name_of_save) # where the model saves the callbacks
        val_check_interval=len(train_loader)
        bar = LitProgressBar()
        
        trainer = pl.Trainer(logger=logger,max_epochs = range_epoch, callbacks=[val_checkpoint_best_loss, val_checkpoint_best_acc, latest_checkpoint, val_checkpoint_acc,val_checkpoint_loss,early_stop],accelerator='gpu',devices=1)
        trainer.fit(model, train_loader, val_loader)
#------------------------------------------------------------


#main parametrs
graph_type = "visual" #"visual", "MTF", "MTF_on_visual", "visual_on_MTF", graph topology
classif = "graph" #"graph", "node", set the type of classification
len_type = "un/cut" #"un/cut", "random", the shape of data used in later paths 
#paths
path_main = "dataset_100_cut.csv" #"dataset_uncut.csv", "dataset_cut.csv", "dataset_rss.npz", paths used for cut/uncut/random dataset
path_properties = "dataset_properties.npz"  # path to properties used for random 
path_mask = "dataset_mask.npz" # path to mask dataset used for random

# params for 
monitor_test = "best_acc.ckpt"
learning_rate = 0.01
batch_size = 64*2
range_epoch = 500 #set length of epoch
save_file="tb_logs"
name_of_save = "100"
#-------------------------------------------------------------

# Training

**EPOCH**  
MTF == 0  
VG == 1000  

MTF_on_VG == 2000  
VG_on_MTF == 3000  

graph == 0  
node == 100  

uncut == 0  
random == 10  

50 == 20  
100 == 30  
150 == 40  
200 == 50  
250 == 60  

In [ ]:
graph_type = "MTF" #"visual", "MTF", "MTF_on_visual", "visual_on_MTF", graph topology
classif = "graph" #"graph", "node", set the type of classification
len_type = "random" #"un/cut", "random", the shape of data used in later paths 

path_main = "dataset_rss.npz" #"dataset_uncut.csv", "dataset_cut.csv", "dataset_rss.npz", paths used for cut/uncut/random dataset
path_properties = "dataset_properties.npz"  # path to properties used for random 
path_mask = "dataset_mask.npz" # path to mask dataset used for random

learning_rate = 0.01
batch_size = 64*2
range_epoch = 1000 #set length of epoch
save_file="Models"
name_of_save = "test"
patience = 200
analysis = True
#-------------------------------------------------------------
for i in range(6): 
    SEED = 10+i
    main()

In [ ]:
graph_type = "visual" #"visual", "MTF", "MTF_on_visual", "visual_on_MTF", graph topology
classif = "node" #"graph", "node", set the type of classification
len_type = "un/cut" #"un/cut", "random", the shape of data used in later paths 

path_main = "dataset_uncut.csv" #"dataset_uncut.csv", "dataset_cut.csv", "dataset_rss.npz", paths used for cut/uncut/random dataset
path_properties = "dataset_properties.npz"  # path to properties used for random 
path_mask = "dataset_mask.npz" # path to mask dataset used for random

learning_rate = 0.01
batch_size = 64*2
range_epoch = 1000 #set length of epoch
save_file="Models"
name_of_save = "VG_node_classification"
patience = 200
analysis = True
#-------------------------------------------------------------
for i in range(6): 
    SEED = 1100+i
    main()

In [ ]:
graph_type = "MTF" #"visual", "MTF", "MTF_on_visual", "visual_on_MTF", graph topology
classif = "node" #"graph", "node", set the type of classification
len_type = "un/cut" #"un/cut", "random", the shape of data used in later paths 

path_main = "dataset_uncut.csv" #"dataset_uncut.csv", "dataset_cut.csv", "dataset_rss.npz", paths used for cut/uncut/random dataset
path_properties = "dataset_properties.npz"  # path to properties used for random 
path_mask = "dataset_mask.npz" # path to mask dataset used for random

learning_rate = 0.01
batch_size = 64*2
range_epoch = 1000 #set length of epoch
save_file="Models"
name_of_save = "MTF_node_classification"
patience = 200
analysis = True
#-------------------------------------------------------------
for i in range(5): 
    SEED = 100+i
    main()

In [ ]:
graph_type = "visual_on_MTF" #"visual", "MTF", "MTF_on_visual", "visual_on_MTF", graph topology
classif = "node" #"graph", "node", set the type of classification
len_type = "un/cut" #"un/cut", "random", the shape of data used in later paths 

path_main = "dataset_uncut.csv" #"dataset_uncut.csv", "dataset_cut.csv", "dataset_rss.npz", paths used for cut/uncut/random dataset
path_properties = "dataset_properties.npz"  # path to properties used for random 
path_mask = "dataset_mask.npz" # path to mask dataset used for random

learning_rate = 0.01
batch_size = 64
range_epoch = 1000 #set length of epoch
save_file="Models"
name_of_save = "VG_on_MTF_node_classification"
patience = 200
analysis = True
#-------------------------------------------------------------
for i in range(6): 
    SEED = 3100+i
    main()

3100


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


node classification on un/cut time series using visual_on_MTF graphs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type    | Params
----------------------------------
0 | conv1 | GATConv | 512   
1 | lin1  | Linear  | 256   
2 | conv2 | GATConv | 16.8 K
3 | lin2  | Linear  | 16.5 K
4 | conv3 | GATConv | 781   
5 | lin3  | Linear  | 129   
----------------------------------
35.0 K    Trainable params
0         Non-trainable params
35.0 K    Total params
0.140     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
graph_type = "MTF_on_visual" #"visual", "MTF", "MTF_on_visual", "visual_on_MTF", graph topology
classif = "node" #"graph", "node", set the type of classification
len_type = "un/cut" #"un/cut", "random", the shape of data used in later paths 

path_main = "dataset_uncut.csv" #"dataset_uncut.csv", "dataset_cut.csv", "dataset_rss.npz", paths used for cut/uncut/random dataset
path_properties = "dataset_properties.npz"  # path to properties used for random 
path_mask = "dataset_mask.npz" # path to mask dataset used for random

learning_rate = 0.01
batch_size = 64
range_epoch = 1000 #set length of epoch
save_file="Models"
name_of_save = "MTF_on_VG_node_classification"
patience = 200
analysis = True
#-------------------------------------------------------------
for i in range(6): 
    SEED = 2100+i
    main()

## ------------------------------------

In [ ]:
graph_type = "visual" #"visual", "MTF", "MTF_on_visual", "visual_on_MTF", graph topology
classif = "graph" #"graph", "node", set the type of classification
len_type = "un/cut" #"un/cut", "random", the shape of data used in later paths 

path_main = "dataset_50_cut.csv" #"dataset_uncut.csv", "dataset_cut.csv", "dataset_rss.npz", paths used for cut/uncut/random dataset

learning_rate = 0.01
batch_size = 64*2
range_epoch = 1000 #set length of epoch
save_file="Models"
name_of_save = "VG_50"
patience = 200
analysis = True
#-------------------------------------------------------------
for i in range(6): 
    SEED = 1020+i
    main()

In [ ]:
graph_type = "visual" #"visual", "MTF", "MTF_on_visual", "visual_on_MTF", graph topology
classif = "graph" #"graph", "node", set the type of classification
len_type = "un/cut" #"un/cut", "random", the shape of data used in later paths 

path_main = "dataset_100_cut.csv" #"dataset_uncut.csv", "dataset_cut.csv", "dataset_rss.npz", paths used for cut/uncut/random dataset

learning_rate = 0.01
batch_size = 64*2
range_epoch = 1000 #set length of epoch
save_file="Models"
name_of_save = "VG_100"
patience = 200
analysis = True
#-------------------------------------------------------------
for i in range(6): 
    SEED = 1030+i
    main()

In [ ]:
graph_type = "visual" #"visual", "MTF", "MTF_on_visual", "visual_on_MTF", graph topology
classif = "graph" #"graph", "node", set the type of classification
len_type = "un/cut" #"un/cut", "random", the shape of data used in later paths 

path_main = "dataset_150_cut.csv" #"dataset_uncut.csv", "dataset_cut.csv", "dataset_rss.npz", paths used for cut/uncut/random dataset

learning_rate = 0.01
batch_size = 64*2
range_epoch = 1000 #set length of epoch
save_file="Models"
name_of_save = "VG_150"
patience = 200
analysis = True
#-------------------------------------------------------------
for i in range(6): 
    SEED = 1040+i
    main()

In [ ]:
graph_type = "visual" #"visual", "MTF", "MTF_on_visual", "visual_on_MTF", graph topology
classif = "graph" #"graph", "node", set the type of classification
len_type = "un/cut" #"un/cut", "random", the shape of data used in later paths 

path_main = "dataset_200_cut.csv" #"dataset_uncut.csv", "dataset_cut.csv", "dataset_rss.npz", paths used for cut/uncut/random dataset

learning_rate = 0.01
batch_size = 64*2
range_epoch = 1000 #set length of epoch
save_file="Models"
name_of_save = "VG_200"
patience = 200
analysis = True
#-------------------------------------------------------------
for i in range(6): 
    SEED = 1050+i
    main()

In [ ]:
graph_type = "visual" #"visual", "MTF", "MTF_on_visual", "visual_on_MTF", graph topology
classif = "graph" #"graph", "node", set the type of classification
len_type = "un/cut" #"un/cut", "random", the shape of data used in later paths 

path_main = "dataset_250_cut.csv" #"dataset_uncut.csv", "dataset_cut.csv", "dataset_rss.npz", paths used for cut/uncut/random dataset

learning_rate = 0.01
batch_size = 64*2
range_epoch = 1000 #set length of epoch
save_file="Models"
name_of_save = "VG_250"
patience = 200
analysis = True
#-------------------------------------------------------------
for i in range(6): 
    SEED = 1060+i
    main()

## ------------------------------------

In [ ]:
graph_type = "MTF" #"visual", "MTF", "MTF_on_visual", "visual_on_MTF", graph topology
classif = "graph" #"graph", "node", set the type of classification
len_type = "un/cut" #"un/cut", "random", the shape of data used in later paths 

path_main = "dataset_50_cut.csv" #"dataset_uncut.csv", "dataset_cut.csv", "dataset_rss.npz", paths used for cut/uncut/random dataset

learning_rate = 0.01
batch_size = 64*2
range_epoch = 1000 #set length of epoch
save_file="Models"
name_of_save = "MTF_50"
patience = 200
analysis = True
#-------------------------------------------------------------
for i in range(6): 
    SEED = 20+i
    main()

In [ ]:
graph_type = "MTF" #"visual", "MTF", "MTF_on_visual", "visual_on_MTF", graph topology
classif = "graph" #"graph", "node", set the type of classification
len_type = "un/cut" #"un/cut", "random", the shape of data used in later paths 

path_main = "dataset_100_cut.csv" #"dataset_uncut.csv", "dataset_cut.csv", "dataset_rss.npz", paths used for cut/uncut/random dataset

learning_rate = 0.01
batch_size = 64*2
range_epoch = 1000 #set length of epoch
save_file="Models"
name_of_save = "MTF_100"
patience = 200
analysis = True
#-------------------------------------------------------------
for i in range(6): 
    SEED = 30+i
    main()

In [ ]:
graph_type = "MTF" #"visual", "MTF", "MTF_on_visual", "visual_on_MTF", graph topology
classif = "graph" #"graph", "node", set the type of classification
len_type = "un/cut" #"un/cut", "random", the shape of data used in later paths 

path_main = "dataset_150_cut.csv" #"dataset_uncut.csv", "dataset_cut.csv", "dataset_rss.npz", paths used for cut/uncut/random dataset

learning_rate = 0.01
batch_size = 64*2
range_epoch = 1000 #set length of epoch
save_file="Models"
name_of_save = "MTF_150"
patience = 200
analysis = True
#-------------------------------------------------------------
for i in range(6): 
    SEED = 40+i
    main()

In [ ]:
graph_type = "MTF" #"visual", "MTF", "MTF_on_visual", "visual_on_MTF", graph topology
classif = "graph" #"graph", "node", set the type of classification
len_type = "un/cut" #"un/cut", "random", the shape of data used in later paths 

path_main = "dataset_200_cut.csv" #"dataset_uncut.csv", "dataset_cut.csv", "dataset_rss.npz", paths used for cut/uncut/random dataset

learning_rate = 0.01
batch_size = 64*2
range_epoch = 1000 #set length of epoch
save_file="Models"
name_of_save = "MTF_200"
patience = 200
analysis = True
#-------------------------------------------------------------
for i in range(6): 
    SEED = 50+i
    main()

In [ ]:
graph_type = "MTF" #"visual", "MTF", "MTF_on_visual", "visual_on_MTF", graph topology
classif = "graph" #"graph", "node", set the type of classification
len_type = "un/cut" #"un/cut", "random", the shape of data used in later paths 

path_main = "dataset_250_cut.csv" #"dataset_uncut.csv", "dataset_cut.csv", "dataset_rss.npz", paths used for cut/uncut/random dataset

learning_rate = 0.01
batch_size = 64*2
range_epoch = 1000 #set length of epoch
save_file="Models"
name_of_save = "MTF_250"
patience = 200
analysis = True
#-------------------------------------------------------------
for i in range(6): 
    SEED = 60+i
    main()

# testing

In [ ]:
#node anomaly location
def anomaly_location_in_words():
    my_model=my_model12
    mm1 = []
    mm2 = []
    points = 0
    if any(my_model) == 1:

        my_model1 = my_model.reshape(150, -1)
        for i in my_model1:
            mm1 = np.append(mm1,all(i))
        if any(mm1) == 1:
            if my_model[-1] != 0:
                my_model2 = my_model.reshape(2, -1)
                for i in my_model2:
                    mm2 = np.append(mm2,all(i))
                if any(mm2) == 1:
                    print("SlowD")
                    return
                print("SuddenD")
                return
            print("SuddenR")
            return
        print("InstaD")
        return
    print("No_anomaly")
    return

In [ ]:
version = "MTF_random"
monitor_test = "best_acc.ckpt"
graph_type = "MTF" #"visual", "MTF", "MTF_on_visual", "visual_on_MTF", graph topology
classif = "graph" #"graph", "node", set the type of classification
len_type = "random" #"un/cut", "random", the shape of data used in later paths 
path_main = "dataset_rss.npz" #"dataset_uncut.csv", "dataset_cut.csv", "dataset_rss.npz", paths used for cut/uncut/random dataset
SEED = 10

my_test_loop()